<a href="https://colab.research.google.com/github/bonillahermes/Data_Science_Projects/blob/main/Clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hermes Yate Bonilla
**Data Scientist**
---

**Contact:**
- **Email:** [bonillahermes@gmail.com](mailto:bonillahermes@gmail.com)
- **LinkedIn:** [linkedin.com/in/bonillahermes](https://www.linkedin.com/in/bonillahermes/)
- **GitHub:** [github.com/bonillahermes](https://github.com/bonillahermes)
- **Webpage:** [bonillahermes.com](https://bonillahermes.com/)
---

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster
from sklearn.cluster import SpectralClustering
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
from sklearn.datasets import load_digits
from scipy.stats import f_oneway

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = "/content/drive/My Drive/Bases/DatosSenImput.xlsx"
df = pd.read_excel(path)

In [ ]:
# Primero, excluimos la columna 'ID' del análisis pero la guardamos para usarla más adelante si es necesario
id_data = df['ID']
analysis_data = df.drop(columns=['ID'])

# PCA

El Análisis de Componentes Principales (PCA, por sus siglas en inglés) es una técnica estadística de reducción de dimensionalidad que se utiliza ampliamente para simplificar la complejidad en conjuntos de datos de alta dimensión al transformarlos a un nuevo sistema de coordenadas de menor dimensión. Las nuevas dimensiones, denominadas componentes principales, se generan de tal manera que el primer componente principal captura la mayor varianza posible de los datos, y cada componente subsiguiente, en orden de extracción, tiene la mayor varianza posible bajo la restricción de ser ortogonal a los anteriores.

La metodología del PCA implica un proceso matemático que transforma las variables originales correlacionadas en un conjunto de variables no correlacionadas. Este proceso comienza con la estandarización de las variables de escala para cada dimensión del conjunto de datos. Luego, se calcula la matriz de covarianza o correlación para identificar las relaciones entre ellas. A partir de esta matriz, se obtienen los autovalores y autovectores que reflejan la magnitud y la dirección de la varianza en los datos. Los autovectores se convierten en los componentes principales, y los autovalores indican la cantidad de varianza que cada componente principal retiene.

La utilidad del PCA en un entorno profesional es multiforme. En el contexto del análisis exploratorio de datos, permite identificar patrones y visualizar la estructura de los datos de manera simplificada. En el ámbito de la construcción de modelos predictivos, el PCA se utiliza para mitigar problemas de multicolinealidad y reducir el tiempo computacional, seleccionando solo aquellos componentes que aportan la mayoría de la información. Además, en campos como la bioinformática y la investigación de mercado, el PCA se emplea para agrupar y clasificar datos complejos de forma intuitiva.

In [ ]:
# Aplicar PCA
pca = PCA(n_components=5)
datos_pca = pca.fit_transform(analysis_data)

In [ ]:
# Varianza explicada por cada componente
varianza_explicada = pca.explained_variance_ratio_
varianza_acumulada = np.cumsum(varianza_explicada)

In [ ]:
print(varianza_explicada)
print(varianza_acumulada)

In [ ]:
# Crear gráfico de la varianza explicada acumulada
plt.figure(figsize=(10, 5))
plt.bar(range(1, len(varianza_explicada) + 1), varianza_explicada, alpha=0.5, align='center', label='Varianza explicada individual')
plt.step(range(1, len(varianza_acumulada) + 1), varianza_acumulada, where='mid', label='Varianza explicada acumulada')
plt.ylabel('Porcentaje de varianza explicada')
plt.xlabel('Componentes principales')
plt.legend(loc='best')
plt.title('Análisis de Componentes Principales - Varianza Explicada')
plt.axhline(y=0.95, color='r', linestyle='--', label='95% varianza explicada')
plt.show()

In [ ]:
# Aplicar PCA con 5 componentes
pca = PCA(n_components=5)
pca.fit(analysis_data)
datos_pca = pca.transform(analysis_data)

# Obtener las cargas (loadings) de PCA
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)

# Crear un DataFrame para los loadings y los nombres de las variables
loadings_df = pd.DataFrame(loadings, index=analysis_data.columns, columns=[f'Component {i+1}' for i in range(5)])

# Crear una lista para almacenar la asignación de variables a componentes
component_assignments = []

# Iterar sobre cada variable y su loading en cada componente
for variable in loadings_df.index:
    for component in loadings_df.columns:
        loading = loadings_df.loc[variable, component]
        if abs(loading) >= 0.4:  # Criterio de umbral
            component_assignments.append((variable, component, loading))

# Convertir la lista en un DataFrame
component_assignments_df = pd.DataFrame(component_assignments, columns=['Variable', 'Component', 'Loading'])

# Mostrar los resultados
print(component_assignments_df)

In [ ]:
component_assignments_df.to_excel('component_assignments_df.xlsx', engine='openpyxl')  # Guardar en un archivo CSV si se necesita

In [ ]:
# Crear un DataFrame con los puntajes de PCA y los IDs correspondientes
scores_df = pd.DataFrame(datos_pca, columns=[f'Component {i+1}' for i in range(5)])
scores_df['ID'] = id_data

# Función para encontrar los individuos más representativos para cada componente
def top_contributors(component, n=5):
    return scores_df[['ID', component]].sort_values(by=component, ascending=False).head(n)

# Aplicar la función para cada componente y almacenar en un diccionario
top_contributors_by_component = {f'Component {i+1}': top_contributors(f'Component {i+1}') for i in range(5)}

# Mostrar los resultados
for component, top_contributors_df in top_contributors_by_component.items():
    print(f'Top contributors for {component}:')
    print(top_contributors_df, '\n')

## Kmeans

### Determinación de números de clusters

In [ ]:
# Calcular la suma de las distancias al cuadrado para un rango de número de clusters
inertia = []
for n in range(1, 10):
    kmeans = KMeans(n_clusters=n, random_state=42).fit(datos_pca)
    inertia.append(kmeans.inertia_)

# Graficar el método del codo
plt.figure(figsize=(20, 10))
plt.plot(range(1, 10), inertia, marker='o')
plt.title('Método del Codo para Determinar el Número Óptimo de Clusters')
plt.xlabel('Número de Clusters')
plt.ylabel('Inertia')
plt.xticks(range(1, 10))
plt.grid(True)
plt.show()

En el gráfico del método del codo, se puede observar cómo disminuye la inercia a medida que aumenta el número de clusters. Lo que se busca en un gráfico de este tipo es un cambio en la tasa de disminución que se asemeje a un "codo". En el gráfico, este punto no es extremadamente pronunciado, pero parece que podría haber un codo alrededor del cluster 3 o 4. Después de este punto, la inercia sigue disminuyendo pero a un ritmo más lento, lo que sugiere que añadir más clusters más allá de este punto no mejora la suma de distancias cuadradas dentro de los clusters.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)  # Ajustar K-means
clusters = kmeans.fit_predict(datos_pca)

In [ ]:
# Crear DataFrame para los resultados
resultados_pca = pd.DataFrame(datos_pca, columns=[f'PC{i+1}' for i in range(5)])
resultados_pca['Cluster'] = clusters
resultados_pca['ID'] = id_data

# Guardar en formato Excel
resultados_pca.to_excel('resultados_pca_clusters.xlsx', index=False)

In [ ]:
# Crear DataFrame para los componentes principales
pca_df = pd.DataFrame(datos_pca, columns=[f'PC{i+1}' for i in range(5)])

# Añadir los componentes y los clusters al DataFrame original
df_final = df.join(pca_df)
df_final['Cluster'] = clusters

# Guardar en formato Excel
df_final.to_excel('df_con_pca_clusters.xlsx', index=False)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(datos_pca[:, 0], datos_pca[:, 1], c=clusters, cmap='viridis')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('Clustering con PCA')
plt.colorbar()
plt.show()

In [ ]:
# Configurar PCA para 3 componentes principales
pca = PCA(n_components=3)
datos_pca_3d = pca.fit_transform(analysis_data)

# Aplicar K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(datos_pca_3d)

In [ ]:
# Crear un gráfico de dispersión 3D interactivo con plotly
fig = go.Figure(data=[go.Scatter3d(
    x=datos_pca_3d[:, 0],
    y=datos_pca_3d[:, 1],
    z=datos_pca_3d[:, 2],
    mode='markers',
    text=id_data,  # Utilizar id_data para los tooltips
    hoverinfo='text+name',  # Mostrar el texto (ID) en los tooltips
    marker=dict(
        size=5,
        color=clusters,  # Asignar colores de acuerdo a las etiquetas de cluster
        colorscale='Viridis',  # Escala de color de Plotly
        opacity=0.8
    )
)])

# Actualizar el diseño del gráfico para añadir título y etiquetas
fig.update_layout(
    title='Clustering con PCA en 3D',
    scene=dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'
    ),
    legend_title="Clusters"
)

# Mostrar el gráfico
fig.show()


In [ ]:
# Filtrar por el clúster 1
mask_cluster1 = clusters == 0
fig1 = go.Figure(data=[go.Scatter3d(
    x=datos_pca_3d[mask_cluster1, 0],
    y=datos_pca_3d[mask_cluster1, 1],
    z=datos_pca_3d[mask_cluster1, 2],
    mode='markers',
    text=np.array(id_data)[mask_cluster1],  # Asumiendo que id_data es una lista o array
    hoverinfo='text+name',
    marker=dict(
        size=5,
        color='purple',  # Color estático para el clúster 1
        opacity=0.8
    )
)])

fig1.update_layout(
    title='Clúster 1 con PCA en 3D',
    scene=dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'
    )
)
fig1.show()


In [ ]:
# Filtrar por el clúster 2
mask_cluster1 = clusters == 1
fig1 = go.Figure(data=[go.Scatter3d(
    x=datos_pca_3d[mask_cluster1, 0],
    y=datos_pca_3d[mask_cluster1, 1],
    z=datos_pca_3d[mask_cluster1, 2],
    mode='markers',
    text=np.array(id_data)[mask_cluster1],  # Asumiendo que id_data es una lista o array
    hoverinfo='text+name',
    marker=dict(
        size=5,
        color='skyblue',  # Color estático para el clúster 2
        opacity=0.8
    )
)])

fig1.update_layout(
    title='Clúster 1 con PCA en 3D',
    scene=dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'
    )
)
fig1.show()

In [ ]:
# Filtrar por el clúster 3
mask_cluster1 = clusters == 2
fig1 = go.Figure(data=[go.Scatter3d(
    x=datos_pca_3d[mask_cluster1, 0],
    y=datos_pca_3d[mask_cluster1, 1],
    z=datos_pca_3d[mask_cluster1, 2],
    mode='markers',
    text=np.array(id_data)[mask_cluster1],  # Asumiendo que id_data es una lista o array
    hoverinfo='text+name',
    marker=dict(
        size=5,
        color='yellow',  # Color estático para el clúster 1
        opacity=0.8
    )
)])

fig1.update_layout(
    title='Clúster 1 con PCA en 3D',
    scene=dict(
        xaxis_title='Componente Principal 1',
        yaxis_title='Componente Principal 2',
        zaxis_title='Componente Principal 3'
    )
)
fig1.show()

In [ ]:
# Añadir las etiquetas de los clusters al DataFrame de puntajes PCA
scores_df = pd.DataFrame(datos_pca, columns=[f'Component {i+1}' for i in range(5)])
scores_df['Cluster'] = clusters

# Calcular los centros medios de cada cluster para cada componente
cluster_centers_df = scores_df.groupby('Cluster').mean()

In [ ]:
# Visualización de los centros de los clusters en cada componente
cluster_centers_df.plot(kind='bar', figsize=(14, 7))
plt.ylabel('Valor Medio del Componente')
plt.xlabel('Cluster')
plt.legend(title='Componentes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(12, 7))
for i in range(1, 6):
    plt.subplot(2, 3, i)
    sns.boxplot(x='Cluster', y=f'Component {i}', data=scores_df)
    plt.title(f'Distribución del Componente {i} por Cluster')
plt.tight_layout()
plt.show()

In [ ]:
descriptive_stats = scores_df.groupby('Cluster').describe().stack(level=0)[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]

# Imprime las estadísticas descriptivas para revisarlas
print(descriptive_stats)

In [ ]:
# Crear un DataFrame con las componentes principales y las etiquetas de clúster
df_pca = pd.DataFrame(datos_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
df_pca['cluster'] = clusters

# Realizar ANOVA para cada componente principal
results = {}
for i in range(1, 6):  # Asumiendo que hay 3 componentes principales
    groups = [df_pca[df_pca['cluster'] == k][f'PC{i}'] for k in range(3)]
    f_value, p_value = f_oneway(*groups)
    results[f'PC{i}'] = (f_value, p_value)

# Imprimir los resultados del ANOVA
for component, (f_val, p_val) in results.items():
    print(f'{component} - F-Value: {f_val}, P-Value: {p_val}')
    if p_val < 0.05:
        print(f"Existen diferencias estadísticamente significativas para {component} entre los grupos.")
    else:
        print(f"No hay diferencias estadísticamente significativas para {component} entre los grupos.")


## t-SNE

In [ ]:
# Rango de perplejidades para probar
perplexities = [5, 10, 30, 50]

for perplexity in perplexities:
    tsne = TSNE(n_components=3, verbose=1, perplexity=perplexity, n_iter=3000, random_state=42)
    Y = tsne.fit_transform(analysis_data)

    plt.figure(figsize=(6, 4))
    plt.scatter(Y[:, 0], Y[:, 1], c='blue')
    plt.title(f't-SNE with Perplexity={perplexity}')
    plt.xlabel('Component 1')
    plt.ylabel('Component 2')
    plt.show()

In [ ]:
# Configurar t-SNE con una semilla para obtener resultados reproducibles
tsne = TSNE(n_components=3, verbose=1, perplexity=5, n_iter=5000, random_state=42)
datos_tsne = tsne.fit_transform(analysis_data)

In [ ]:
# Gráfico de los datos en 2D
plt.figure(figsize=(10, 6))
plt.scatter(datos_tsne[:, 0], datos_tsne[:, 1])
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.title('Visualización t-SNE')
plt.grid(True)
plt.show()

## Kmeans

In [ ]:
# datos_tsne contiene los datos de t-SNE
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(datos_tsne)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 8))
plt.plot(range(1, 11), wcss)
plt.title('El Método del Codo')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()


In [ ]:
#Kmeans
kmeans = KMeans(n_clusters=3)  # Ajustar el número de clusters según la necesidad del análisis
cluster_labels_kmeans = kmeans.fit_predict(datos_tsne)  # Realizar el clustering

# Visualizar los clusters
plt.figure(figsize=(10, 6))
plt.scatter(datos_tsne[:, 0], datos_tsne[:, 1], c=cluster_labels_kmeans, cmap='viridis', s=50, alpha=0.6, edgecolor='k')  # s: tamaño de punto, alpha: transparencia, edgecolor: color del borde de los puntos
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.title('Clusters K-Means sobre t-SNE')
plt.colorbar()  # Mostrar la leyenda de colores para los clusters
plt.grid(True)
plt.show()

In [ ]:
# Crear un gráfico de dispersión 3D interactivo con plotly
fig = go.Figure(data=[go.Scatter3d(
    x=datos_tsne[:, 0],
    y=datos_tsne[:, 1],
    z=datos_tsne[:, 2],
    mode='markers',
    text=id_data,  # Usar id_data para los tooltips
    hoverinfo='text+name',  # Mostrar el texto (ID) y el nombre del trace (útil para identificar los clusters)
    marker=dict(
        size=5,
        color=cluster_labels_kmeans,  # Asignar colores de acuerdo a las etiquetas de cluster
        colorscale='Viridis',  # Escala de color de Plotly
        opacity=0.6,
        line=dict(
            color='Black',  # Color de los bordes de los puntos
            width=0.5       # Ancho de los bordes
        )
    )
)])

# Actualizar el diseño del gráfico para añadir título y etiquetas
fig.update_layout(
    title='Clusters K-Means sobre t-SNE en 3D',
    scene=dict(
        xaxis_title='t-SNE feature 1',
        yaxis_title='t-SNE feature 2',
        zaxis_title='t-SNE feature 3'
    ),
    legend_title="Clusters"
)

# Mostrar el gráfico
fig.show()


## DBSCAN

In [ ]:
# 'eps' y 'min_samples' son parámetros clave que pueden necesitar ajustes basados en la densidad y distribución de los datos
dbscan = DBSCAN(eps=0.6, min_samples=3)
cluster_labels_dbscan = dbscan.fit_predict(datos_tsne)  # Realizar el clustering con los datos de t-SNE

# Visualizar los clusters
plt.figure(figsize=(10, 6))
plt.scatter(datos_tsne[:, 0], datos_tsne[:, 1], c=cluster_labels_dbscan, cmap='viridis', s=50, alpha=0.5, edgecolor='k')  # Mejoras visuales
plt.xlabel('t-SNE feature 1')
plt.ylabel('t-SNE feature 2')
plt.title('Clusters DBSCAN sobre t-SNE')
plt.colorbar(label='Cluster Label')  # Aclarar que la barra de colores representa las etiquetas de los clusters
plt.grid(True)
plt.show()

# UMAP

In [ ]:
!pip install umap-learn

In [ ]:
import umap

In [ ]:
# Definir rangos para los parámetros
neighbor_space = np.arange(5, 50, 15)  # Explora desde 5 vecinos hasta 50, en pasos de 15
min_dist_space = np.arange(0.1, 0.6, 0.2)  # Explora desde 0.1 hasta 0.5, en pasos de 0.2

fig, axs = plt.subplots(len(neighbor_space), len(min_dist_space), figsize=(15, 12))

for i, n_neighbors in enumerate(neighbor_space):
    for j, min_dist in enumerate(min_dist_space):
        umap_reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, random_state=42, n_components=2)
        embedding = umap_reducer.fit_transform(analysis_data)
        ax = axs[i, j]
        scatter = ax.scatter(embedding[:, 0], embedding[:, 1], c='blue', cmap='Spectral', s=5)
        ax.set_title(f"Neighbors: {n_neighbors}, Min_dist: {min_dist}")
        ax.xaxis.set_major_formatter(plt.NullFormatter())
        ax.yaxis.set_major_formatter(plt.NullFormatter())

plt.tight_layout()
plt.show()

In [ ]:
# Configurar UMAP
reducer = umap.UMAP(n_neighbors=20, min_dist=0.30, random_state=42, n_components=3)
datos_umap = reducer.fit_transform(analysis_data)  # Usar datos sin la columna 'ID'

In [ ]:
# Gráfico de los datos en 2D
plt.figure(figsize=(10, 6))
plt.scatter(datos_umap[:, 0], datos_umap[:, 1], alpha=0.5, cmap='viridis', s=50, edgecolor='k')  # s: tamaño de punto, alpha: transparencia, edgecolor: color del borde de los puntos
plt.xlabel('UMAP feature 1')
plt.ylabel('UMAP feature 2')
plt.title('Visualización UMAP de los Datos')
plt.grid(True)
plt.colorbar(label='Cluster Label')  # Aclarar que la barra de colores representa las etiquetas de los clusters, si aplicable
plt.show()

## Kmeans

In [ ]:
# datos_tsne contiene tus datos de t-SNE
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(datos_umap)
    wcss.append(kmeans.inertia_)

plt.figure(figsize=(10, 8))
plt.plot(range(1, 11), wcss)
plt.title('El Método del Codo')
plt.xlabel('Número de clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Asumiendo que 'datos_umap' contiene los resultados de UMAP
kmeans = KMeans(n_clusters=2)  # Define el número de clusters adecuado
cluster_labels = kmeans.fit_predict(datos_umap)  # Realizar el clustering

# Visualizar los clusters
plt.figure(figsize=(10, 6))
plt.scatter(datos_umap[:, 0], datos_umap[:, 1], c=cluster_labels, cmap='viridis', alpha=0.5)
plt.title('Clusters K-Means sobre UMAP')
plt.xlabel('UMAP feature 1')
plt.ylabel('UMAP feature 2')
plt.colorbar()  # Mostrar la leyenda de colores para los clusters
plt.grid(True)
plt.show()

In [ ]:
# Crear un gráfico de dispersión 3D interactivo con plotly
fig = go.Figure(data=[go.Scatter3d(
    x=datos_umap[:, 0],
    y=datos_umap[:, 1],
    z=datos_umap[:, 2],
    mode='markers',
    text=id_data,  # Utilizar id_data para los tooltips
    hoverinfo='text',  # Mostrar el texto (ID) en los tooltips
    marker=dict(
        size=5,
        color=cluster_labels,  # Asignar colores de acuerdo a las etiquetas de cluster
        colorscale='Viridis',  # Escala de color de Plotly
        opacity=0.8
    )
)])

# Actualizar el diseño del gráfico para añadir título y etiquetas
fig.update_layout(
    title='Clusters K-Means sobre UMAP en 3D',
    scene=dict(
        xaxis_title='UMAP feature 1',
        yaxis_title='UMAP feature 2',
        zaxis_title='UMAP feature 3'
    ),
    legend_title="Clusters"
)

# Mostrar el gráfico
fig.show()




## DBSCAN

In [ ]:
# Ajustar los parámetros de DBSCAN según sea necesario
dbscan = DBSCAN(eps=0.6, min_samples=5)  # Ajusta 'eps' y 'min_samples' según tus datos
dbscan_labels = dbscan.fit_predict(datos_umap)  # Realizar el clustering

# Visualizar los clusters
plt.figure(figsize=(10, 6))
plt.scatter(datos_umap[:, 0], datos_umap[:, 1], c=dbscan_labels, cmap='viridis', alpha=0.5)
plt.title('Clusters DBSCAN sobre UMAP')
plt.xlabel('UMAP feature 1')
plt.ylabel('UMAP feature 2')
plt.colorbar()  # Mostrar la leyenda de colores para los clusters
plt.grid(True)
plt.show()



# Clustering Jerarquico

El clustering jerárquico es una técnica de análisis de datos utilizada para detectar y agrupar patrones o conjuntos de observaciones dentro de un gran conjunto de datos. La metodología subyacente del clustering jerárquico no requiere la especificación previa del número de clusters a identificar, a diferencia de otros métodos de clustering como K-Means.

Este método construye una jerarquía de clusters, que se visualiza comúnmente en un dendrograma. El dendrograma es una representación gráfica que muestra la secuencia de fusiones o divisiones de clusters a lo largo del proceso de análisis. Cada fusión se representa con una bifurcación en el dendrograma, donde la altura de la bifurcación refleja la distancia o disimilitud entre los clusters que se están uniendo.

Existen dos enfoques principales en el clustering jerárquico: el aglomerativo y el divisivo. El enfoque aglomerativo, el más común, comienza tratando cada observación como un cluster individual y fusiona sucesivamente los pares de clusters más cercanos hasta que todas las observaciones están contenidas en un solo cluster. Por otro lado, el enfoque divisivo comienza con todas las observaciones en un solo cluster y procede dividiendo sucesivamente los clusters hasta que cada observación forma su propio cluster.

El resultado del clustering jerárquico proporciona no solo una partición de los datos, sino también una representación de las observaciones que refleja tanto las estructuras anidadas como la relación entre los grupos a diferentes niveles de similitud. Este aspecto lo hace particularmente útil en áreas donde las estructuras de los datos son intrínsecamente jerárquicas, como en la biología para la clasificación de especies, en la organización de bibliotecas de documentos o en la segmentación de mercados en marketing.

En la práctica profesional, el clustering jerárquico es apreciado por su flexibilidad y la riqueza interpretativa de sus resultados. La capacidad de visualizar y evaluar la cohesión y separación de los datos a diferentes niveles de agrupamiento facilita la toma de decisiones basadas en una comprensión detallada de las estructuras de datos subyacentes.

In [ ]:
# Enlazar usando el método de Ward
linked = linkage(datos_pca, 'ward')

# Dibujar el dendrograma
plt.figure(figsize=(10, 7))
dendrogram(linked, orientation='top', distance_sort='descending', show_leaf_counts=True)
plt.title('Dendrograma de Clustering Jerárquico')
plt.xlabel('Número de muestra')
plt.ylabel('Distancia')
plt.show()

In [ ]:
# Queremos cortar el dendrograma a una distancia de 100
clusters = fcluster(linked, t=48, criterion='distance')

# Añadir los labels de los clusters al DataFrame original
df['Cluster_Label'] = clusters

# Ahora 'df' contiene una nueva columna 'Cluster_Label' que indica a qué cluster pertenece cada observación

In [ ]:
# Calcular estadísticas descriptivas para cada cluster
cluster_stats = df.groupby('Cluster_Label').agg(['mean', 'median', 'std', 'min', 'max'])

# Ahora, puedes imprimir o explorar 'cluster_stats' para obtener una visión general de cada cluster
print(cluster_stats)

In [ ]:
# Para visualizar las diferencias entre los clusters, seleccionemos una característica para ver cómo se distribuye
# Reemplaza 'feature_to_plot' con el nombre real de tu característica
feature_to_plot = 'IR9'

# Crear un boxplot para cada cluster
plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster_Label', y=feature_to_plot, data=df)
plt.title('Distribución de la característica por Cluster')
plt.show()

In [ ]:
# Diagrama de violín que proporciona más información sobre la densidad
plt.figure(figsize=(10, 6))
sns.violinplot(x='Cluster_Label', y=feature_to_plot, data=df)
plt.title('Distribución de la característica por Cluster')
plt.show()

In [ ]:
# Lista para guardar los resultados del ANOVA
anova_results = []

# Iterar sobre cada columna numérica para realizar ANOVA
for column in analysis_data.columns:
    # Preparar los datos para ANOVA
    group_data = [df[df['Cluster_Label'] == label][column] for label in df['Cluster_Label'].unique()]

    # Realizar ANOVA y guardar el p-valor
    f_val, p_val = stats.f_oneway(*group_data)
    anova_results.append({'Variable': column, 'F-Value': f_val, 'P-Value': p_val})

# Convertir los resultados en un DataFrame
anova_df = pd.DataFrame(anova_results)

# Añadir una columna que indica si las diferencias son significativas
alpha = 0.05  # Nivel de significancia
anova_df['Significant Difference'] = anova_df['P-Value'] < alpha

# Mostrar los resultados
anova_df

# Análisis Factorial

El Análisis Factorial es una técnica estadística utilizada para describir la variabilidad entre variables observadas correlacionadas en términos de un número potencialmente menor de variables no observadas, llamadas factores. La idea principal es que las observaciones influenciadas por un conjunto de factores subyacentes pueden representar la información esencial en un espacio de dimensiones reducido.

En el análisis factorial, se asume que las correlaciones entre las variables observadas se deben a su comunalidad, es decir, a su compartición de uno o más factores comunes. El proceso implica estimar la carga de cada factor, que representa la contribución de cada factor a la variable observada, y la varianza única, que es la parte de la varianza que es única para la variable y no es compartida con otros factores.

### Diferencias entre PCA y Análisis Factorial:

### Objetivo:
- PCA: Orientado a la maximización de la varianza total explicada por los componentes extraídos. Se utiliza para reducir la dimensionalidad manteniendo la mayor cantidad posible de la variabilidad original.
- Análisis Factorial: Busca modelar la correlación entre las variables en términos de un número de factores subyacentes. Está más enfocado en identificar la estructura subyacente y los constructos latentes que explican las correlaciones entre las variables.

### Modelo Estadístico:
- PCA: No distingue entre varianza común y única; cada componente principal incluye una parte de la varianza total (común y única).
- Análisis Factorial: Se diferencia entre varianza común (compartida entre variables y atribuible a los factores) y varianza única (específica de cada variable).

### Interpretación de Componentes/Factores:

- PCA: Los componentes son combinaciones lineales de las variables observadas y pueden no ser interpretables.
- Análisis Factorial: Los factores están pensados para ser interpretados y etiquetados como constructos latentes.

### Rotación:

- PCA: Los componentes son ortogonales entre sí y no están sujetos a rotación.
- Análisis Factorial: Se permite la rotación de los factores para mejorar la interpretabilidad, que puede ser ortogonal o oblicua, permitiendo la correlación entre los factores.

In [ ]:
!pip install factor_analyzer

In [ ]:
from factor_analyzer import FactorAnalyzer


In [ ]:
# Inicializar el objeto FactorAnalyzer con el número de factores que deseas extraer y el método de rotación
fa = FactorAnalyzer(n_factors=5, rotation='varimax', method='principal')
fa.fit(analysis_data)

In [ ]:
# Obtener la varianza explicada de cada factor
factor_variance = fa.get_factor_variance()

# Crear un DataFrame para mostrar la varianza explicada
variance_df = pd.DataFrame({
    'Total Variance': factor_variance[0],
    'Proportional Variance': factor_variance[1],
    'Cumulative Variance': factor_variance[2]
}, index=[f'Factor {i+1}' for i in range(5)])  # Se indexa desde 1 a 5 para incluir los primeros 5 factores

print(variance_df)


In [ ]:
# Obtener la varianza de cada factor
ev, v = fa.get_eigenvalues()

In [ ]:
# Crear un scree plot para visualizar la varianza explicada por cada factor
plt.figure(figsize=(10, 7))  # Ajustar el tamaño del gráfico para una mejor visualización
plt.scatter(range(1, analysis_data.shape[1]+1), ev)
plt.plot(range(1, analysis_data.shape[1]+1), ev)
plt.title('Scree Plot para Análisis Factorial')
plt.xlabel('Número de Factores')
plt.ylabel('Valor Propio (Eigenvalue)')
plt.axhline(y=1, color='r', linestyle='--')  # Línea para el criterio de Kaiser
plt.text(5, 1.05, 'Criterio de Kaiser', color = 'red')  # Texto para explicar la línea de corte de Kaiser
plt.grid(True)
plt.show()

 Basado en este scree plot, un punto de partida sería considerar entre 5 y 6 factores para el análisis subsiguiente. Sin embargo, esto debe ser informado por un entendimiento más profundo de los datos y el contexto en el que se aplicará el análisis factorial.

In [ ]:
# Inicializar el objeto FactorAnalyzer con 5 factores y rotación varimax
fa = FactorAnalyzer(n_factors=5, rotation='varimax')
fa.fit(analysis_data)

# Obtener las cargas factoriales (loadings) de las variables
loadings = fa.loadings_

# Crear un DataFrame para los loadings y los nombres de las variables
loadings_df = pd.DataFrame(loadings, index=analysis_data.columns, columns=[f'Factor {i+1}' for i in range(5)])

# Crear una función para identificar el factor dominante para cada variable
def identify_dominant_factor(loadings, threshold=0.4):
    dominant_factor = None
    max_loading = threshold
    for i, loading in enumerate(loadings):
        if abs(loading) >= max_loading:
            dominant_factor = i + 1  # Factor numbering starts at 1
            max_loading = abs(loading)
    return dominant_factor

# Aplicar la función a cada fila y guardar los resultados en un DataFrame separado
factor_assignments = pd.DataFrame(index=loadings_df.index, columns=['Factor', 'Loading'])
for variable in loadings_df.index:
    dominant_factor = identify_dominant_factor(loadings_df.loc[variable], threshold=0.4)
    if dominant_factor:
        factor_assignments.loc[variable, 'Factor'] = f'Factor {dominant_factor}'
        factor_assignments.loc[variable, 'Loading'] = loadings_df.loc[variable, f'Factor {dominant_factor}']


In [ ]:
# Muestra las primeras filas del resultado
print(factor_assignments)

In [ ]:
factor_assignments.to_excel('factor_assignments.xlsx', engine='openpyxl')